# ANÁLISIS EXPLORATORIO DE LOS DATOS (EDA)

---
## 1. IMPORTAR PAQUETES
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.graph_objects as go

pd.options.display.float_format = '{:.2f}'.format

%config IPCompleter.greedy=True # Activa el autocompletar avanzado en Jupyter

sns.set_theme(style="whitegrid")

---
## 2. IMPORTAR DATOS
---

In [ ]:
# Cargar el DataFrame desde pickle
df = pd.read_pickle("../data/processed/ecommerce_transformado.pkl")
df

---
## 3. SOBRE EL CUSTOMER JOURNAY 
---


---
### ANALIZANDO LA VARIABLE EVENTOS - VISIÓN GENERAL 
---
- El objetivo es analizar cuales son los pasos que da un cliente desde que llega a la web, visualiza productos y termina comprando

In [ ]:
# Contar la cantidad de cada tipo de evento
eventos = df.evento.value_counts()

# Calcular el porcentaje de visualizaciones (view) sobre el total de eventos
porcentaje_view = (eventos['view'] / eventos.sum()) * 100

# Total de visualizaciones (view)
visualizaciones = 100

# Calcular el porcentaje de productos añadidos al carrito (cart) sobre las visualizaciones (view)
porcentaje_cart_sobre_view = (eventos['cart'] / eventos['view']) * 100

# Calcular la tasa de productos sacados del carrito (remove_from_cart) sobre los añadidos al carrito (cart)
porcentaje_remove_sobre_cart = (eventos['remove_from_cart'] / eventos['cart']) * 100

# Calcular la tasa de productos comprados (purchase) sobre los añadidos al carrito (cart)
porcentaje_purchase_sobre_cart = (eventos['purchase'] / eventos['cart']) * 100


# Crear el DataFrame con los valores proporcionados
data = {
    'kpi': ['visualizaciones', 'carrito', 'abandono', 'compra'],
    'valor': [visualizaciones, porcentaje_cart_sobre_view, porcentaje_remove_sobre_cart, porcentaje_purchase_sobre_cart],  
}

kpis = pd.DataFrame(data)

# Mostrar los resultados
print(f"Porcentaje de visualizaciones (view) sobre el total de eventos: {porcentaje_view:.2f}%")
print(f"\nPorcentaje de productos añadidos al carrito (cart) sobre el total de visualizaciones (view): {porcentaje_cart_sobre_view:.2f}%")
print(f"Porcentaje de productos sacados del carrito (remove_from_cart) sobre añadidos al carrito (cart): {porcentaje_remove_sobre_cart:.2f}%")
print(f"Porcentaje de productos comprados (purchase) sobre añadidos al carrito (cart): {porcentaje_purchase_sobre_cart:.2f}%")
display(kpis)

### Visualizamos la información en gráficos de funnel 

In [ ]:
# Esta función recibe la variable de los eventos como entrada
def funnel_analytics_1(evento):
    
    # hacemos el conteo
    eventos = df.evento.value_counts()

    # preparamos las variables
    visualizaciones = 100
    porcentaje_cart_sobre_view = (eventos['cart'] / eventos['view']) * 100
    porcentaje_remove_sobre_cart = (eventos['remove_from_cart'] / eventos['cart']) * 100
    porcentaje_purchase_sobre_cart = (eventos['purchase'] / eventos['cart']) * 100
    
    kpis = pd.DataFrame({'kpi':['visualizaciones','carrito','compra'],
                         'valor':[visualizaciones, porcentaje_cart_sobre_view, porcentaje_purchase_sobre_cart]})

    # Crear el gráfico
    fig = go.Figure(go.Funnel(
    y = kpis['kpi'],  # Nombres de las etapas
    x = kpis['valor'].round(2),  # Valores en porcentaje
    marker = {'color': ['red', 'blue', 'green']},  # Colores personalizados
    opacity = 0.3,  # Opacidad de las barras
    textinfo = "value+percent initial"  # Mostrar valores y porcentaje inicial
    ))

    # Configurar el título del gráfico
    fig.update_layout(title = "- Porcentaje de productos añadidos al carrito sobre el total de visualizaciones <br>- Porcentaje de productos comprados sobre añadidos al carrito")

    # Mostrar el gráfico
    fig.show()

    #imprimimos un informe de conclusiones
    print(f'Porcentaje de productos añadidos al carrito (cart) sobre el total de visualizaciones (view): {porcentaje_cart_sobre_view:.2f}% ')
    print(f"Porcentaje de productos comprados (purchase) sobre añadidos al carrito (cart): {porcentaje_purchase_sobre_cart:.2f}%")
    print(f'\nPor tanto existe un {100 - porcentaje_cart_sobre_view.round(2)}% de visualizaciones sobre las que trabajar para conseguir más carritos, y un {100 - porcentaje_purchase_sobre_cart.round(2)}% de carritos sobre los que trabajar para conseguir más compras.')

funnel_analytics_1(df.evento)

In [ ]:
# Esta función recibe la variable de los eventos como entrada
def funnel_analytics_2(evento):
    
    # hacemos el conteo
    eventos = df.evento.value_counts()

    # preparamos las variables
    carrito = 100
    porcentaje_remove_sobre_cart = (eventos['remove_from_cart'] / eventos['cart']) * 100
    porcentaje_purchase_sobre_cart = (eventos['purchase'] / eventos['cart']) * 100
    
    kpis = pd.DataFrame({'kpi':['carrito','abandono','compra'],
                         'valor':[carrito, porcentaje_remove_sobre_cart, porcentaje_purchase_sobre_cart]})
    
    # Crear el gráfico
    fig = go.Figure(go.Funnel(
    y = kpis['kpi'],  # Nombres de las etapas
    x = kpis['valor'].round(2),  # Valores en porcentaje
    marker = {'color': ['red', 'blue', 'green']},  # Colores personalizados
    opacity = 0.3,  # Opacidad de las barras
    textinfo = "value+percent initial"  # Mostrar valores y porcentaje inicial
    ))

    # Configurar el título del gráfico
    fig.update_layout(title = "- Porcentaje de productos sacados del carrito sobre añadidos al carrito <br>- Porcentaje de productos comprados sobre añadidos al carrito")

    # Mostrar el gráfico
    fig.show()

    #imprimimos un informe de conclusiones
    print(f'Porcentaje de productos sacados del carrito (remove_from_cart) sobre el total de añadidos al carrito (cart): {porcentaje_remove_sobre_cart:.2f}%')
    print(f"Porcentaje de productos comprados (purchase) sobre el total de añadidos al carrito (cart): {porcentaje_purchase_sobre_cart:.2f}%")
    print(f'\nPor tanto existe un {porcentaje_remove_sobre_cart.round(2)}% de abandonos sobre los que se puede trabajar')

funnel_analytics_2(df.evento)

---
### Calculo de la facturacion media por mes
---

In [ ]:
facturacion_por_mes = df.loc[df.evento == 'purchase'].groupby('mes').precio.sum()
print(f"Facturación por mes: \n{facturacion_por_mes}\n")

# Calcular la facturación media por mes
facturacion_media = facturacion_por_mes.mean()
print(f"Facturación media por mes: {facturacion_media:.2f}\n")

---
### Media de cada evento por sesión
---

In [ ]:
# Agrupar por 'sesion_usuario' y 'evento', y contar los productos
eventos_por_sesion = df.groupby(['sesion', 'evento'], observed=True).producto.count().unstack().fillna(0)

# Calcular la media de cada tipo de evento por sesión
media_eventos_sesion = eventos_por_sesion.mean()

# Reordenar la salida en el orden deseado
orden = ['view', 'cart', 'remove_from_cart', 'purchase']
media_eventos_sesion = media_eventos_sesion.reindex(orden)

# Mostrar los resultados
print("Media de eventos por sesión (ordenados):")
print(media_eventos_sesion)
print("\nSe deben incrementar la media de visualizaciones, carritos y compras por sesión y disminuir los abandonos de carrito.")

---
### Visualizaciones de eventos por hora
---

In [ ]:
# Agrupar por 'hora' y 'evento', y contar los eventos
eventos_por_hora = df.groupby(['hora', 'evento'], observed = True).size().unstack(fill_value=0)

# Crear el gráfico de líneas para todos los eventos
plt.figure(figsize=(12, 6))
for evento in eventos_por_hora.columns:
    plt.plot(eventos_por_hora.index, eventos_por_hora[evento], label=evento)

# Configurar el título y las etiquetas
plt.title("Total de eventos por hora", fontsize=16)
plt.xlabel("Hora", fontsize=12)
plt.ylabel("Cantidad de eventos", fontsize=12)
plt.xticks(range(0, 24))  # Asegurar que las horas estén en el rango 0-23
plt.legend(title="Eventos", fontsize=10)
plt.grid(alpha=0.3)

# Mostrar el DataFrame resultante
print("Total de eventos por hora:\n")
print(eventos_por_hora);

---
### Porcentaje de compras sobre visitas por hora
---
- Localizamos las horas donde tenemos mayor y menor tasa de conversión

In [ ]:
# Calcular la proporción de compras sobre visitas por hora
eventos_por_hora['compras_visitas'] = eventos_por_hora['purchase'] / eventos_por_hora['view'] * 100

# Ordenadar las variables
eventos_por_hora = eventos_por_hora[['view', 'cart', 'remove_from_cart', 'purchase', 'compras_visitas']]

print(eventos_por_hora)

# Configurar el estilo de Seaborn
sns.set_theme(style="whitegrid")

# Crear el gráfico de líneas
plt.figure(figsize=(12, 6))
sns.lineplot(
    x=eventos_por_hora.index,  # Eje X: las horas
    y=eventos_por_hora['compras_visitas'],  # Eje Y: porcentaje de compras sobre visitas
    marker='o',  # Marcadores en los puntos
    color='blue',  # Color de la línea
    label='Compras sobre visitas (%)'
)

# Configurar el título y las etiquetas
plt.title("Porcentaje de Compras sobre Visitas por Hora", fontsize=16)
plt.xlabel("Hora", fontsize=12)
plt.ylabel("Porcentaje (%)", fontsize=12)
plt.xticks(range(0, 24))  # Asegurar que las horas estén en el rango 0-23
plt.legend(fontsize=10)
plt.grid(alpha=0.3);

* Las horas en las que la gente compra más son la 1, las 8, de 11 a 13 y las 18 por lo que estos horarios seria buen momento para hacer campañas de publicidad o retargeting
* Las horas en las que la gente no compra son las 24, de 3 a 7, de 14 a 17 y de 19 a 23

---
### Analisis semanal de los eventos
---

In [ ]:
# Analizar la tendencia de eventos por semana
tendencia = df.groupby('evento', observed = True).resample('W').evento.count().unstack(level = 0)

# Ordenar las columnas de acuerdo a la secuencia de eventos
tendencia = tendencia[['view', 'cart', 'remove_from_cart', 'purchase']]

# Mostrar el análisis semanal
print("Tendencia semanal de eventos:\n")
print(tendencia)

# Crear el gráfico de líneas para la tendencia semanal
tendencia.plot(subplots = True, figsize=(16, 8), 
               xticks = tendencia.index, 
               x_compat = True, 
               rot = 90, 
               title="Tendencia Semanal de Eventos", 
               marker='o');

- Durante el mes de diciembre disminuyen notablemente todos los valores de los eventos, posiblemente el motivo es por que los clientes compran en Black Friday que es la ultima semana de Noviembre y se puede ver claramente que es donde esta el pico mas alto de todos los valores de los eventos.

---
### Analisis diario de los eventos de Noviembre y Diciembre
---

In [ ]:

# Filtrar los datos para los meses de noviembre y diciembre
tendencia_diaria_nov_dic = df.query("'2019-11-01' <= fecha <= '2019-12-31'").groupby('evento', observed = True).resample('D').evento.count().unstack(level = 0)

# Ordenar las columnas de acuerdo a la secuencia de eventos
tendencia_diaria_nov_dic = tendencia_diaria_nov_dic[['view', 'cart', 'remove_from_cart', 'purchase']]

# Mostrar el análisis diario
print("Análisis diario de los eventos en noviembre y diciembre:")
print(tendencia_diaria_nov_dic)

# Crear el gráfico de líneas para la tendencia diaria
tendencia_diaria_nov_dic.plot(subplots = True, figsize=(16, 8), 
                       xticks = tendencia_diaria_nov_dic.index, 
                       x_compat = True, 
                       rot = 90, 
                       title="Tendencia Diaria de Eventos (Noviembre y Diciembre)", 
                       marker='o');

* El pico coincide con el black friday (día 29)
* Pero aún hay un pico mayor unos días antes, el día 22, posiblemente por el inicio de la semana black friday
* Los días de Navidad tienen una tendencia decreciente, lo que significa que los consumidores claramente han adelantado sus compras

---
### Analisis diario de los eventos de Enero y Febrero
---

In [ ]:
# Filtrar los datos para los meses de Enero y Febrero
tendencia_diaria_ene_feb = df.query("'2020-01-01' <= fecha <= '2020-02-29'").groupby('evento', observed = True).resample('D').evento.count().unstack(level = 0)

# Ordenar las columnas de acuerdo a la secuencia de eventos
tendencia_diaria_ene_feb = tendencia_diaria_ene_feb[['view', 'cart', 'remove_from_cart', 'purchase']]

# Mostrar el análisis diario
print("Análisis diario de los eventos en enero y febrero:")
print(tendencia_diaria_ene_feb)

# Crear el gráfico de líneas para la tendencia diaria
tendencia_diaria_ene_feb.plot(subplots = True, figsize=(16, 8), 
                                      xticks = tendencia_diaria_ene_feb.index, 
                                      x_compat = True, 
                                      rot = 90, 
                                      title="Tendencia Diaria de Eventos (Enero y Febrero)", 
                                      marker='o');


* Durante la semana de Reyes no hay pico de ventas, ni los días previos a San Valentín, pero sí hay un pico muy pronunciado el 27 de Enero, seguramente algún evento local seguramente coincide con la celebracion del año nuevo chino en Rusia.


**➢ INSIGHT** La gran conclusión es que la mayor parte de las compras navideñas se reparte en la semana del black friday

---
### Analisis por día y hora de las compras
---

In [ ]:
# Agrupar por fecha y hora
#compras_dia_hora = df.loc[df.evento == 'purchase'].groupby(['date', 'hora']).size().unstack(level=0).fillna(0)
compras_dia_hora = df.loc[df.evento == 'purchase'].groupby(['date', 'hora']).evento.count().unstack(level=0).fillna(0)

# Mostrar la tabla
display(compras_dia_hora)

# Crear el gráfico de calor
plt.figure(figsize=(16, 8))
sns.heatmap(compras_dia_hora)
plt.title("Compras por Hora y Día", fontsize=16)
plt.xlabel("date", fontsize=12)
plt.ylabel("hora", fontsize=12)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout();


---
## 3. SOBRE LOS CLIENTES
---

In [ ]:
df

- Creamos un DataFrame a nivel de clientes

In [ ]:
# Filtramos todos los registros que sean eventos de compra (purchase) para descartar los usuarios que no han realizado compras
clientes = df.query("evento == 'purchase'").groupby('usuario').agg({'producto':'count', # número total de productos que el usuario ha comprado
                                                         'sesion':'nunique', # número de veces que el usuario ha comprado, cada sesion es una compra, aunque en una misma sesion compre varios productos
                                                         'precio':'mean', # precio medio de los productos comprados por el usuario
                                                         'date':'max'}) # ultima fecha en la que el usuario ha realizado una compra

# Renombramos las columnas
clientes.rename(columns={'producto':'num_productos_comprados', 
                          'sesion':'num_compras_totales', 
                          'precio':'precio_medio_producto', 
                          'date':'ult_compra'}, inplace=True)

# Agregando nuevas columnas
clientes['gasto_total'] = clientes.num_productos_comprados * clientes.precio_medio_producto # total gastado por el cliente
clientes['productos_por_compra'] = clientes.num_productos_comprados / clientes.num_compras_totales # número de productos comprados por compra

clientes

### Como se distribuyen los clientes con respecto al gasto total

In [ ]:

# Configurar el estilo de Seaborn
sns.set_theme(style="whitegrid")

# Crear un histograma para visualizar la distribución del gasto total
plt.figure(figsize=(12, 4))
sns.histplot(data=clientes, x='gasto_total', bins = 200, kde=True, color='blue', alpha=0.7)
plt.xlim(0, 300)  # Limitar el eje x para enfocarnos en el rango de gasto total

# Configurar el título y las etiquetas
plt.title("Distribución de Clientes por Gasto Total", fontsize=16)
plt.xlabel("Gasto Total", fontsize=12)
plt.ylabel("Número de Clientes", fontsize=12);



- La mayoria de los clientes han gastado menos de 50€ en el periodo

### Como se distribuyen los clientes con relación a la frecuencia de compra

In [ ]:
# Crear un gráfico de barras con countplot para visualizar la distribución de la frecuencia de compra
plt.figure(figsize=(12, 4))
sns.countplot(data = clientes, x = 'num_compras_totales', color = 'blue', alpha = 0.7)

# Configurar el título y las etiquetas
plt.title("Distribución de Clientes por Frecuencia de Compra", fontsize = 16)
plt.xlabel("Número Total de Compras", fontsize = 12)
plt.ylabel("Número de Clientes", fontsize = 12);



- La mayoria de los clienttes realizan solo una compra, existe una opotunidad de mejorar esta palanca fidelizando clientes lo que incrementaria significativamente la facturación

**➢ INSIGHT** La mayoría de los clientes solo hace una compra

### Cuantos productos de media compra un cliente en cada compra

In [ ]:
print(clientes.productos_por_compra.describe())

# crear un grafico de bigotes para ver outliers
plt.figure(figsize=(12, 4))
sns.boxplot(data = clientes, x = 'productos_por_compra', color = 'blue')
plt.xlim(0, 220)  # Limitar el eje x para enfocarnos en el rango de productos por compra

# Configurar el título y las etiquetas
plt.title("Distribución de Clientes por Productos Comprados por Compra", fontsize=16)
plt.xlabel("Número de Productos Comprados por Compra", fontsize=12)
plt.ylabel("Número de Clientes", fontsize=12);


- Hay un 25% de los clientes que realizan mas 10 de compras, lo que quiere decir que los clientes compran, se deben realizar acciones para incrementar el numero medio de productos por compra

**➢ INSIGHT** La compra de la mediana es de 5 productos

### Quienes son los clientes que mas han comprado o gastado

In [ ]:
# Filtrar los 10 clientes con mayor gasto total
display(clientes.nlargest(n = 10, columns = 'gasto_total'))

# Explorar la estadística descriptiva del gasto total
clientes.gasto_total.describe()

- **➢ INSIGHT** Existen clientes que su gasto esta muy por encima de la media, pueden ser clientes que compran para revender el producto, si la empresa no tiene en cuenta este tipo de cliente esta palanca es importante para crear algun tipo de estrtategia para dirigerse a clientes profesionales.

### ¿Cual es la supervivencia de los clientes?

In [ ]:
# Preparamos un dataframe solo con compradores y con las variables usuario y mes.
c = df.loc[df.evento == 'purchase', ['usuario','mes']]
display(c)

# Pasamos los meses a columnas
c = pd.crosstab(c.usuario,c.mes).reset_index()
display(c)

In [ ]:
# Renombramos y eliminamos el usuario, c1 = oct, c2 = nov, c3 = dic, c4 = ene, c5 = feb
c.columns = ['usuario','c4','c5','c1','c2','c3']
c.drop(columns = 'usuario', inplace = True)
c

In [ ]:
# establecemos la base / clientes que en el mes 1 no han tenido ninguna compra / clientes q en el mes 2 han tenido alguna compra
c2 = c.loc[(c.c1 == 0) & (c.c2 > 0)]
print(c2)

# Pasamos a un dataframe binario ya que solo nos importa si ese cliente ha comprado o no en cada mes
def binarizar(variable):
    variable = variable.transform(lambda x: 1 if (x > 0) else 0)
    return(variable)

c2_b = c2.apply(binarizar)
print(c2_b)

# Calculamos la media de supervivientes en cada uno de los meses posteriores
c2_f = c2_b.sum() / c2_b.shape[0]
c2_f = c2_f.sort_index()
(c2_f)

In [ ]:
# establecemos la base / clientes que en el mes 2 no han tenido ninguna compra / clientes q en el mes 3 han tenido alguna compra
# Pasamos a un dataframe binario ya que solo nos importa si ese cliente ha comprado o no en cada mes
# Calculamos la media de supervivientes en cada uno de los meses posteriores
c3 = c.loc[(c.c2 == 0) & (c.c3 > 0)]
c3_b = c3.apply(binarizar)
c3_f = c3_b.sum() / c3_b.shape[0]
c3_f = c3_f.sort_index()
c3_f['c1'] = 0
c3_f


In [ ]:
# establecemos la base / clientes que en el mes 3 no han tenido ninguna compra / clientes q en el mes 4 han tenido alguna compra
# Pasamos a un dataframe binario ya que solo nos importa si ese cliente ha comprado o no en cada mes
# Calculamos la media de supervivientes en cada uno de los meses posteriores
c4 = c.loc[(c.c3 == 0) & (c.c4 > 0)]
c4_b = c4.apply(binarizar)
c4_f = c4_b.sum() / c4_b.shape[0]
c4_f = c4_f.sort_index()
c4_f['c1'] = 0
c4_f['c2'] = 0
c4_f


In [ ]:
# Creamos el DataFrame
cor = pd.DataFrame({'c2':c2_f,'c3':c3_f,'c4':c4_f})
print(cor)


cor = cor.drop(index = 'c1').T
print(cor)


plt.figure(figsize = (12,6))
sns.heatmap(cor,annot = True, fmt = '.0%', cmap='Greys');

**INSIGHT**: El 90% de que los nuevos clientes no vuelve a comprar en los meses posteriores


In [ ]:
df

---
## 3. SOBRE LOS PRODUCTOS
---

In [ ]:
# Creamos un dataframe a nivel de producto para poder analizar esta dimensión.
# calculamos los conteos de cada evento en cada producto.

prod = df.groupby(['producto','evento']).size()
prod  = prod.unstack(level = 1).fillna(0).reset_index()
prod

In [ ]:
# Vamos a incorporar el precio, para ello primero creamos un maestro de precios por producto.

maestro_precios = df.groupby('producto', as_index = False).precio.mean()
print(maestro_precios)



In [ ]:

prod = pd.merge(left = prod, right = maestro_precios, how = 'left', on = 'producto')
print(prod)

In [ ]:
# reordenamos nombres
prod = prod[['producto','view','cart','remove_from_cart','purchase','precio']]
prod

### ¿Cuales son los productos mas vendidos?

In [ ]:
prod.sort_values('purchase',ascending = False)[0:20]

### ¿Hay productos que no se venden?

In [ ]:
prod[prod.purchase == 0]

INSIGHT: Casi la mitad de los productos no han tenido ninguna venta en los 5 meses del histórico.

### Relación entre el precio y el volumen de ventas

In [ ]:
sns.scatterplot(data = prod[prod.purchase > 0], x = 'precio', y = 'purchase', hue = 'precio');

### Relación entre el precio por debajo de 50€ y el volumen de ventas

In [ ]:
sns.scatterplot(data = prod[(prod.purchase > 0) & (prod.precio < 50)], x = 'precio', y = 'purchase', hue = 'precio');

### Productos que se eliminan del carrito

In [ ]:
prod.insert(loc = 4,
            column = 'remove_from_cart_porc',
            value = prod.remove_from_cart / prod.cart *100 )
prod

### Cuales son los productos mas vistos

In [ ]:
prod.view.sort_values(ascending = False)[0:20].plot.bar();

### Relacion de productos con muchas vistas y pocas compras

In [ ]:
sns.scatterplot(data = prod.loc[prod.view < 4000], x = 'view', y = 'purchase', hue = 'precio')
plt.xlim(1000,3000)
plt.ylim(0,150);

En cada sesión, de media:

* KPIs por sesión: Se ven 2.2 productos
* KPIs por sesión: Se añaden 1.3 productos al carrito
* KPIs por sesión: Se eliminan 0.9 productos del carrito
* KPIs por sesión: Se compran 0.3 productos
* Recurrencia: el 10% de los clientes vuelve a comprar tras el primer mes
* Conversión: 60% de añadir al carrito sobre visualizaciones
* Conversión: 22% de compra sobre añadidos a carrito
* Conversión: 13% de compra sobre visualizaciones
* Facturación media mensual: 125.000€